In [1]:
import pandas as pd
import numpy as np
import intake

import geopandas as gpd
import folium

In [2]:
cat = intake.open_catalog('../catalogs/catalog.yml')
measures = cat.measures.read().set_index('cdtfa_id')

In [3]:
us_counties = (gpd.read_file('https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_county_5m.zip')
              .set_index('GEOID'))

In [4]:
measures['county_fips'] = measures['county_fips'].apply(lambda x: '0' + str(x))
measures = measures.set_index('county_fips')

In [5]:
lost_geo = us_counties.join(measures, how='inner')[['NAME', 'geometry', 'year']].dropna()

In [6]:
lost_geo

,NAME,geometry,year
06001,Alameda,"POLYGON ((-122.33371 37.80980, -122.32357 37.8...",2014.0
06013,Contra Costa,"POLYGON ((-122.42976 37.96540, -122.41536 37.9...",2004.0
06019,Fresno,"POLYGON ((-120.65595 36.95283, -120.59057 36.9...",2007.0
06025,Imperial,"POLYGON ((-116.10563 32.72157, -116.10315 32.7...",2008.0
06037,Los Angeles,"MULTIPOLYGON (((-118.60337 33.47810, -118.5987...",1980.0
06037,Los Angeles,"MULTIPOLYGON (((-118.60337 33.47810, -118.5987...",1990.0
06037,Los Angeles,"MULTIPOLYGON (((-118.60337 33.47810, -118.5987...",2008.0
06037,Los Angeles,"MULTIPOLYGON (((-118.60337 33.47810, -118.5987...",2016.0
06039,Madera,"POLYGON ((-120.54170 37.04451, -120.47669 37.0...",2006.0
06041,Marin,"MULTIPOLYGON (((-122.44632 37.86105, -122.4385...",2018.0


In [7]:
lost_geo = lost_geo.sort_values('year', ascending=True).drop_duplicates('NAME').sort_index()

In [8]:
m = folium.Map(location=[36.5, -121], zoom_start=6, tiles='Stamen Toner')

folium.Choropleth(
    geo_data=lost_geo,
    name="choropleth",
    data=lost_geo,
    columns=["NAME","year"],
    key_on="feature.properties.NAME",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Year First LOST Introduced",
).add_to(m)

m